In [51]:
%run ConfigFile.ipynb
%run DB_Connection.ipynb

import pyodbc
import copy
import math
from datetime import datetime
import logging
import time
import os
import sys
import ast
import json

import pandas as pd
import numpy as np
import re

# Text preprocessing imports
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
#creating a unique id with the help of current date timestamp
st = datetime.fromtimestamp(time.time()).strftime('%d%m%Y%H%M%S')
clientname=client_name['clientname']
logging.basicConfig(filename=clientname+"_"+st+".log",level=logging.DEBUG,format="%(asctime)s:%(levelname)s:%(message)s")
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger(clientname)

<Logger SAAB (DEBUG)>

In [3]:
today_timestamp = datetime.now()

### Preprocessing

In [4]:
'''
    This function preprocesses the transcript column to remove punctuation, whitespaces and to lowercase & lemmatize them 
    Input:
        data : the entire dataframe on which the preprocessing needs to be performed
    Output:
        preprocessed list of transcripts
'''
def preprocess_text(input_dataframe,target_column_number):
    # Preprocessing text
    lemmatized_text = []
    count = 0
    for row in input_dataframe.itertuples():
#         print(row)
        # Removing extra spaces, special characters and punctuation marks and lower casing the text
        clean_text = re.sub(r'\s\s+',r' ',re.sub(r'[?|$|.|!|#|%|^|*|;|,|+|-|_|=|&]',r'',row[target_column_number].lower()))
        # Lemmatizing the text
        
        lemmatized_text.append(" ".join([str(token.lemma_).replace('-PRON-',str(token)) for token in nlp(str(clean_text))]))
    return lemmatized_text

In [97]:
'''
    This function is for running through all the rows of the agent transcriptsad assign different script adherance scores
    Input:
        data : the entire dataframe on which the preprocessing needs to be performed
    Output:
        Series of scores of all the script adherance for each call
'''
def adherance_check(row_data,adherance):#row_data,
    # Retrieving lsit of keywords for each kind of adherance
    adherance_check_list = {}
    for row in adherance.itertuples():
        for section in ast.literal_eval(row[5]):
            sections_score = {}
            if(section=='Greetings'):
                    row_data.split(":")[:5]
            elif(section=='Closure'):
                row_data.split(":")[5:]
            for phrase in ast.literal_eval(row[5])[section]:
                if phrase in row_data:
                    sections_score[section] = 1

        adherance_check_list[row[3]] = round(len(sections_score)/len(ast.literal_eval(row[5])),2)
        
    return adherance_check_list

### Writing to DW

In [ ]:
'''
    Function to write the dataframe to DW
  
    Input :
        script_adherance_df : dataframe that consists of scores of the scripte adherance checks for each call
            
'''
def write_df_to_dw(script_adherance_df):
    # Writing to DW
    today_timestamp = datetime.now()
    server = 'saab-server-resource.database.windows.net'
    database = 'SAAB_DW_Resource'
    username = 'saabadmin'
    password = 'p@$$w0rd'
    conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = conn.cursor()
    id_count = 0
    for index,row in script_adherance_df.iterrows():
        id_count += 1
        cursor.execute("INSERT INTO [dbo].[SAAB_ML_SCRIPT_ADHERANCE_FT]([RESULT_ID],[DOMAIN_ID],[SCRIPT_ID],[CALL_ID],[SCORE],[CREATED_DATE],[CREATED_BY]) values (?, ?, ?, ?, ?, ?, ?)", id_count, row['DOMAIN_ID'], row['SCRIPT_ID'], row['CALL_ID'], row['SCORE'],row['CREATED_DATE'],row['CREATED_BY']) 
    conn.commit()
    cursor.close()
    conn.close()

### Main function

In [6]:
def main():
    domain_id = 101
    created_by = "MDXC"
    try:
        raw_transcript_data = fetch_data("select * from [dbo].[Fact_Audio_Insights] order by [Call_ID],[StartTime];")
        agent_label_data = fetch_data("select * from [dbo].[SAAB_ML_SPEAKER_MAPPING_FT] order by Result_ID")
        adherance_data = fetch_data("select * from [dbo].[SAAB_ML_MASTER_SCRIPT_ADHERANCE_DM] where DOMAIN_ID = " + str(domain_id))
    except Exception as err:
        if "Could not open a connection to SQL Server" in str(err):
            logging.error("Could not connect to data warehouse : "+str(err.args)+"\nTraceback :"+str(err.with_traceback))
        else:
            logging.error("Error occured with database :"+str(err.args)+"\nTraceback :"+str(err.with_traceback))
        sys.exit(1)
    try:
        # Assign respective agent and customer transcripts
        for row in agent_label_data.itertuples():
            agent_label_data.loc[row[0],'Transcripts'] = ":".join(list(raw_transcript_data[((raw_transcript_data['Call_ID']==row[2]) & (raw_transcript_data['SpeakerId']==row[4]))].Display))
        # Preprocess transcripts
        agent_label_data['Lemmatized_Transcript']  = preprocess_text(agent_label_data,10)
        
        # Scoring for script adherance
        agent_label_data['Adherance_Scores'] = np.where(agent_label_data['LABEL']=='A',agent_label_data['Lemmatized_Transcript'].apply(adherance_check,adherance=adherance_data),"")
        
        # Preparing final script sdherance dataframe
        script_ft_df = pd.concat([pd.DataFrame(agent_label_data[agent_label_data['LABEL']=='A'].DOMAIN_ID),pd.DataFrame(agent_label_data[agent_label_data['LABEL']=='A'].CALL_ID),agent_label_data[agent_label_data['LABEL']=='A'].Adherance_Scores.apply(pd.Series)],axis=1)
        script_ft_df = script_ft_df.melt(id_vars=["DOMAIN_ID","CALL_ID"], var_name="SCRIPT_ID",value_name="SCORE").sort_values('CALL_ID')
        script_ft_df['CREATED_DATE'] = today_timestamp
        script_ft_df['CREATED_BY'] = created_by
        
        # Writing dataframe to DW
        
    except Exception as err:
        

In [77]:
agent_label_data[agent_label_data['LABEL']=='A'].Adherance_Scores.apply(pd.Series)    

,1,2
0,0.00,0.0
2,0.33,1.0
4,0.33,0.0
6,0.33,0.0
8,0.33,0.0
...,...,...
171,0.33,0.0
173,0.33,0.0
175,0.33,0.0
177,0.33,0.0


In [60]:
script_ft = pd.DataFrame()
for row in agent_label_data[agent_label_data['LABEL']=='A'].itertuples():
    for i in range(len(agent_label_data[agent_label_data['LABEL']=='A'].Adherance_Scores[0])):
        
    script_ft['Call']
#     for i in ast.literal_eval(row[11]):
#         print(i)
#         break

Pandas(Index=0, RESULT_ID=1, CALL_ID=56482635, DOMAIN_ID=101, SPEAKER_ID=1, LABEL='A', CREATED_DATE=Timestamp('2020-01-29 17:51:32.620000'), CREATED_BY='Mrinalini', UPDATED_DATE=None, UPDATED_BY=None, Transcripts="Oh yes, so mom, you can check the fun value of your policy under policy summary section.:Sure, mom, you may also register this for the SMS on the move facility where you.:Mom this is just a one time registration pop process.:Oh, fine, I'll do that, OK, ma'am, so you can register for the service by.:Alright.:Please mom uh this is only for a unit link policy. I just wanted to inform you. Yes, I got that.:Is there anything else I may assist you with?", Lemmatized_Transcript='oh yes so mom you can check the fun value of your policy under policy summary section : sure mom you may also register this for the sms on the move facility where you : mom this be just a one time registration pop process : oh fine i will do that ok madam so you can register for the service by : alright : pl

In [30]:
script_ft = pd.DataFrame()
script_ft['DOMAIN_ID'] = domain_id
script_ft['CALL_ID'] = agent_label_data[agent_label_data['LABEL']=='A'].CALL_ID
    
        

,RESULT_ID,CALL_ID,DOMAIN_ID,SPEAKER_ID,LABEL,CREATED_DATE,CREATED_BY,UPDATED_DATE,UPDATED_BY,Transcripts,Lemmatized_Transcript,Adherance_Scores
0,1,56482635,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,"Oh yes, so mom, you can check the fun value of...",oh yes so mom you can check the fun value of y...,"{1: 0.0, 2: 0.0}"
2,3,61876574,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,"Good afternoon, HDFC Life customer care. This ...",good afternoon hdfc life customer care this be...,"{1: 0.33, 2: 1.0}"
4,5,85550179,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,Good morning. Welcome to HDFC insurance? How c...,good morning welcome to hdfc insurance how can...,"{1: 0.33, 2: 0.0}"
6,7,98762981,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,HDFC life customer care this is Alex how may I...,hdfc life customer care this be alex how may i...,"{1: 0.33, 2: 0.0}"
8,9,114965010,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,Graph noon AGC life customer care. This is Ale...,graph noon agc life customer care this be alex...,"{1: 0.33, 2: 0.0}"
...,...,...,...,...,...,...,...,...,...,...,...,...
171,172,1999173881,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,"Good afternoon, HDFC this is Alexa how may I h...",good afternoon hdfc this be alexa how may i he...,"{1: 0.33, 2: 0.0}"
173,174,2039734603,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,Hello this is Alexa from HDFC life? How may I ...,hello this be alexa from hdfc life how may i h...,"{1: 0.33, 2: 0.0}"
175,176,2071978881,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,Good afternoon HDFC life customer care this is...,good afternoon hdfc life customer care this be...,"{1: 0.33, 2: 0.0}"
177,178,2074449487,101,1,A,2020-01-29 17:51:32.620,Mrinalini,None,None,"Good afternoon, HDFC life.:This is Alex how ma...",good afternoon hdfc life : this be alex how ma...,"{1: 0.33, 2: 0.0}"


In [111]:
greetings_list = (adherance_data[adherance_data['SCRIPT_TYPE']=='Greetings'].KEYWORDS.values[0]).replace("'","").replace("}","").replace("{","").split(",")
closure_list = (adherance_data[adherance_data['SCRIPT_TYPE']=='Closure'].KEYWORDS.values[0].replace("'","").replace("}","").replace("{","").split(","))

['how can i help ', ' what can I assist', ' welcome to', ' how may i help', ' you are through to', ' my name is', ' good afternoon', ' thank you for calling', ' what may i assist', ' good morning']
['is there anything else', ' have a great day', ' have a nice day', ' thank you for calling', ' thank you for contacting', ' thanks for calling', ' have a good day']


#### Writing to master table - keywords

In [243]:
master_df = pd.DataFrame()
master_df['master_id'] = [1,2]
# master_df['domain_id'] = [101,101]
master_df['script_id'] = [1,2]
master_df['script_type'] = ['Greetings','Closure']
master_df['keywords'] = ['{"Section_1":["good morning", "good afternoon"],"Section_2":["welcome to", "you are through to","thank you for calling"],"Section_3":["how can i help", "how may i help", "what can I assist", "what may i assist"]}','{"Section_1":["thanks for calling","thank you for calling","have a nice day","have a good day","thank you for contacting", "have a great day", "is there anything else"]}']
# master_df['created_date'] = [today_timestamp,today_timestamp]
# master_df['created_by'] = ["MDXC","MDXC"]
master_df

,master_id,script_id,script_type,keywords
0,1,1,Greetings,"{""Section_1"":[""good morning"", ""good afternoon""..."
1,2,2,Closure,"{""Section_1"":[""thanks for calling"",""thank you ..."


In [244]:
server = 'saab-server-resource.database.windows.net'
database = 'SAAB_DW_Resource'
username = 'saabadmin'
password = 'p@$$w0rd'
conn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
for index,row in master_df.iterrows():
    cursor.execute("INSERT INTO [dbo].[SAAB_ML_MASTER_SCRIPT_ADHERANCE_DM]([MASTER_ID],[DOMAIN_ID],[SCRIPT_ID],[SCRIPT_TYPE],[KEYWORDS],[CREATED_DATE],[CREATED_BY]) values (?, ?, ?, ?, ?, ?, ?)", row['master_id'], 101,row['script_id'], row['script_type'], str(row['keywords']), today_timestamp,'MDXC') 
conn.commit()
cursor.close()
conn.close()